In [1]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import psycopg2
from dotenv import load_dotenv
import time

In [3]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) # Convert list item to a dict
    alt_titles = {} # Initialize a dictionary to hold alt titles
    
    # Column headers that could hold alt titles
    alt_keys = ['Also known as', 'Arabic', 'Cantonese', 'Chinese', 'French', 'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 
                'Literally', 'Mandarin', 'McCune–Reischauer', 'Original title', 'Polish', 'Revised Romanization', 
                'Romanized', 'Russian', 'Simplified', 'Traditional', 'Yiddish']
    
    # Check each key for data
    for key in alt_keys:

        # If data exists in current key, add it to alt titles dict and remove it from movie dict
        if key in movie: 
            alt_titles[key] = movie[key]
            movie.pop(key)

    # If current movie contains alt titles, add them under alt_titles key to original dict
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # Create a function that will combine like-columns based on the tuple "names"
    def change_column_name(names):
        if names[0] in movie:
            movie[names[1]] = movie.pop(names[0])

    # A list of tuples that will be passed to the change_column_name function. First entry is old name, second is new name.
    columns_to_change = [
        ('Adaptation by', 'Writer(s)'),
        ('Screen story by', 'Writer(s)'),
        ('Screenplay by', 'Writer(s)'),
        ('Story by', 'Writer(s)'),
        ('Written by', 'Writer(s)'),
        ('Adaptation by', 'Writer(s)'),
        ('Country of origin', 'Country'),
        ('Directed by','Director(s)'),
        ('Director','Director(s)'),
        ('Created by', 'Creator(s)'),
        ('Distributed by','Distributor(s)'),
        ('Distributor','Distributor(s)'),
        ('Edited by', 'Editor(s)'),
        ('Length', 'Running time'),
        ('Music by', 'Composer(s)'),
        ('Original release', 'Release date'),
        ('Produced by', 'Producer(s)'),
        ('Producer', 'Producer(s)'),
        ('Production company(s)', 'Production company(-ies)'),
        ('Productioncompanies ', 'Production company(-ies)'),
        ('Productioncompany ', 'Production company(-ies)'),
        ('Released', 'Release date'),
        ('Theme music composer', 'Composer(s)')
    ]

    [change_column_name(names) for names in columns_to_change]

    return movie

In [11]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)
def movies_ETL():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv('Resources/movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv('Resources/ratings.csv')

    # Open and read the Wikipedia data JSON file.
    with open('Resources/wikipedia-movies.json', mode='r') as file:
        wiki_movies = json.load(file)
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies if 'No. of episodes' not in movie]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_wiki_movies = [clean_movie(movie) for movie in wiki_movies]

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_wiki_movies)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as e:
        print(e)

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    columns_to_keep = [col for col in wiki_movies_df.columns if wiki_movies_df[col].isnull().sum() < .9 * len(wiki_movies_df)]
    wiki_movies_df = wiki_movies_df[columns_to_keep]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

    # 12. Add the parse_dollars function.
    def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r"\$\s*\d+\.?\d*\s*milli?on", s, flags=re.IGNORECASE):
            # remove the $ and million
            s = re.sub(r'\$|\s|[a-zA-Z]','', s)
            # convert to float and multiply my a million
            s = float(s) * 10 ** 6
            # return value
            return s
        # if input is of the form $###.# billion
        elif re.match(r"\$\s*\d+\.?\d*\s*billi?on", s, flags=re.IGNORECASE):
            # remove the $ and billion
            s = re.sub(r'\$|\s|[a-zA-Z]', '', s)
            # convert to float and multiply my a billion
            s = float(s) * 10 ** 9
            # return value
            return s
        # if input is of the form $###,###,###
        elif re.match(form_two, s):
            # Remove the $ and ,
            s = re.sub(r'\$|,', '', s)
            # convert to float
            s = float(s)
            # return value
            return s
        # otherwise, return NaN
        else:
            return np.nan
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget']
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+[123]?\d,?\s+\d{4}' # Month dd, yyyy
    date_form_two = r'\d{4}.[01]\d.[0123]\d' # yyyy-mm-dd
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}' # Month yyyy
    date_form_four = r'\d{4}' # yyyy
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    runtime = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    runtime_extract = runtime.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    runtime_extract = runtime_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = runtime_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

    # Drop columns whose data has been updated
    wiki_movies_df.drop(columns=['Box office', 'Budget', 'Release date', 'Running time'])
     
    # 2. Clean the Kaggle metadata.
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop(columns='adult', axis=1)
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    kaggle_metadata['budget'] = pd.to_numeric(kaggle_metadata['budget'], errors='raise')
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'], errors='raise')
    # print(kaggle_metadata.dtypes)

    # 3. Merged the two DataFrames into the movies DataFrame.
    movies_df = wiki_movies_df.merge(kaggle_metadata, on='imdb_id', suffixes=['_wiki', '_kaggle'])

    # 4. Drop unnecessary columns from the merged DataFrame.
    movies_df = movies_df.drop(movies_df[(movies_df['release_date_wiki'] > '2005-01-01') & (movies_df['release_date_kaggle'] < '1960-01-01')].index)
    movies_df = movies_df.drop(columns=['title_wiki', 'release_date_wiki', 'Language', 'Production company(-ies)'])
    movies_df.drop(columns='video', inplace=True)

    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(kaggle_column, wiki_column, df=movies_df):

        df[kaggle_column] = df.apply(lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column], axis=1)
        df.drop(columns=wiki_column, inplace=True)

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data('runtime', 'running_time')
    fill_missing_kaggle_data('budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data('revenue', 'box_office')

    # 7. Filter the movies DataFrame for specific columns.
    movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                        'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                        'genres','original_language','overview','spoken_languages','Country',
                        'production_companies','production_countries','Distributor(s)',
                        'Producer(s)','Director(s)','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                        ]]

    # 8. Rename the columns in the movies DataFrame.
    movies_df.rename({'id':'kaggle_id',
                    'title_kaggle':'title',
                    'url':'wikipedia_url',
                    'budget_kaggle':'budget',
                    'release_date_kaggle':'release_date',
                    'Country':'country',
                    'Distributor(s)':'distributor',
                    'Producer(s)':'producers',
                    'Director(s)':'director',
                    'Starring':'starring',
                    'Cinematography':'cinematography',
                    'Editor(s)':'editors',
                    'Writer(s)':'writers',
                    'Composer(s)':'composers',
                    'Based on':'based_on'
                    }, axis='columns', inplace=True)

    # 9. Transform and merge the ratings DataFrame.
    
    
    return wiki_movies_df, movies_with_ratings_df, movies_df


In [12]:
wiki_movies_df, movies_with_ratings_df, movies_df = movies_ETL()
movies_df[(movies_df['release_date_wiki'] > '2005-01-01') & (movies_df['release_date_kaggle'] < '1960-01-01')]

belongs_to_collection            object
budget                            int64
genres                           object
homepage                         object
id                                int64
imdb_id                          object
original_language                object
original_title                   object
overview                         object
popularity                      float64
poster_path                      object
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
spoken_languages                 object
status                           object
tagline                          object
title                            object
video                              bool
vote_average                    float64
vote_count                      float64
dtype: object


,url,year,imdb_link,title_wiki,Based on,Starring,Cinematography,Release date,Running time,Country,...,release_date_kaggle,revenue,runtime,spoken_languages,status,tagline,title_kaggle,video,vote_average,vote_count
3629,https://en.wikipedia.org/wiki/The_Holiday,2006.0,https://www.imdb.com/title/tt00457939/,The Holiday,NaN,"[Kate Winslet, Cameron Diaz, Jude Law, Jack Bl...",Dean Cundey,"[December 8, 2006, (, 2006-12-08, )]",136 minutes,United States,...,1953-08-28,30500000.0,118.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pouring out of impassioned pages...brawling th...,From Here to Eternity,False,7.2,137.0


In [4]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = 'Resources'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia.movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [5]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

In [6]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [7]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

In [8]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

In [9]:
# 15. Check the movies_df DataFrame. 
movies_df.head()